In [1]:
# Parameters
Company_Name = "Finance Helpers"
Company_Investor_Name = "Great Investors"
Domicile_Name = "Jakarta"


In [2]:
#these will be overwritten based on the parameters that the user will write
Company_Name=None
Company_Investor_Name= None
Domicile_Name= None

In [3]:
import requests
import json

In [4]:
unfiltered_endpoint="/api/get/investment"
company_search="Finance Helpers"

url=f"http://localhost:8080{unfiltered_endpoint}"

In [5]:
body=requests.get(url).json()

In [6]:
import pandas as pd
df=pd.DataFrame(body)

In [7]:
df.drop(columns=["_id","__v"],inplace=True, errors='ignore')
# df.head(5)

In [8]:
df["Date_Of_Ownership"]=pd.to_datetime(df["Date_Of_Ownership"])
# df.head(5)

In [9]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

PREDICTION MODEL

In [10]:
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.theta import ThetaForecaster
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
from sktime.utils.plotting import plot_series

from sklearn.metrics import mean_absolute_error, r2_score
from prophet import Prophet
import numpy as np

C:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
df_to_predict= df.loc[(df['Company']== Company_Name) & (df['Company_Investor']== Company_Investor_Name) & (df['Domicile']== Domicile_Name) ]
df_to_predict.head(5)

,Company,Company_Investor,Domicile,Year_Of_Operation,Business,Percentage_Ownership,Revenue,Expense,Ebida,Tax_Investment,Price_Asset,Price_Liability,Equity,COGS,Date_Of_Ownership


In [12]:
df_to_predict=df_to_predict.set_index(['Date_Of_Ownership'])
df_to_predict=df_to_predict.sort_index()
df_to_predict.index

DatetimeIndex([], dtype='datetime64[ns, UTC]', name='Date_Of_Ownership', freq=None)

In [13]:
df_to_predict=df_to_predict.reset_index()

In [14]:
df_to_predict=df_to_predict.drop(["Company","Company_Investor","Domicile","Business","Percentage_Ownership","Year_Of_Operation"], axis=1)
df_to_predict.head(5)

,Date_Of_Ownership,Revenue,Expense,Ebida,Tax_Investment,Price_Asset,Price_Liability,Equity,COGS


In [15]:
df_to_predict=df_to_predict.dropna()
df_to_predict.isna().count()

Date_Of_Ownership    0
Revenue              0
Expense              0
Ebida                0
Tax_Investment       0
Price_Asset          0
Price_Liability      0
Equity               0
COGS                 0
dtype: int64

In [16]:
df_to_predict['Date_Of_Ownership'] = pd.to_datetime(df_to_predict['Date_Of_Ownership']).dt.tz_localize(None)
df_to_predict2=df_to_predict.copy()

In [17]:
df_to_predict.head()

,Date_Of_Ownership,Revenue,Expense,Ebida,Tax_Investment,Price_Asset,Price_Liability,Equity,COGS


In [18]:
days_to_predict = 30

In [19]:
forecast_results = {}

columns_to_forecast= ['Revenue','Expense','Ebida','Tax_Investment','Price_Asset','Price_Liability','Equity','COGS']

for column in columns_to_forecast:
    # Prepare the data for Prophet
    data = df[['Date_Of_Ownership', column]].rename(columns={'Date_Of_Ownership': 'ds', column: 'y'})
    data['ds'] = pd.to_datetime(data['ds']).dt.tz_localize(None)
    cap_value = data['y'].quantile(0.95)
    data['cap'] = cap_value
    
    model= Prophet(growth='logistic', yearly_seasonality=False, weekly_seasonality=True,daily_seasonality=True)
    # model.add_seasonality(name='quarterly', period=30, fourier_order=5)
    model.fit(data)
    
    # Make future dataframe for the next 90 days
    future = model.make_future_dataframe(periods=days_to_predict, include_history=False)
    future['cap']=cap_value
    
    # Forecast the data
    forecast = model.predict(future)
    
    # Store the results
    forecast_results[column] = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


00:01:49 - cmdstanpy - INFO - Chain [1] start processing


00:01:50 - cmdstanpy - INFO - Chain [1] done processing


00:01:51 - cmdstanpy - INFO - Chain [1] start processing


00:01:52 - cmdstanpy - INFO - Chain [1] done processing


00:01:52 - cmdstanpy - INFO - Chain [1] start processing


...

00:01:55 - cmdstanpy - INFO - Chain [1] done processing


00:01:56 - cmdstanpy - INFO - Chain [1] start processing


00:01:56 - cmdstanpy - INFO - Chain [1] done processing


00:01:57 - cmdstanpy - INFO - Chain [1] start processing


00:01:57 - cmdstanpy - INFO - Chain [1] done processing


In [20]:
# Combine forecasts into a single dataframe
combined_forecast = pd.DataFrame({'ds': forecast_results[columns_to_forecast[0]]['ds']})

for column in columns_to_forecast:
    col_forecast = forecast_results[column].rename(columns={
        'yhat': f'{column}_forecast',
        'yhat_lower': f'{column}_lower',
        'yhat_upper': f'{column}_upper'
    })
    combined_forecast = combined_forecast.merge(col_forecast, on='ds')

# Save or view the combined forecast
combined_forecast.head()


,ds,Revenue_forecast,Revenue_lower,Revenue_upper,Expense_forecast,Expense_lower,Expense_upper,Ebida_forecast,Ebida_lower,Ebida_upper,...,Price_Asset_upper,Price_Liability_forecast,Price_Liability_lower,Price_Liability_upper,Equity_forecast,Equity_lower,Equity_upper,COGS_forecast,COGS_lower,COGS_upper
0,2025-01-01,2.624794e+06,818310.559716,4.274492e+06,1.071878e+06,-9984.305792,2.250797e+06,1.298795e+06,-19311.532685,2.602507e+06,...,8.344121e+06,1.799515e+06,72289.650750,3.488825e+06,3.748632e+06,1.196297e+06,6.244568e+06,140984.556950,87942.627405,195708.906624
1,2025-01-02,2.506832e+06,839525.404970,4.177019e+06,9.788655e+05,-154348.738262,2.103899e+06,1.279369e+06,-47871.281110,2.632148e+06,...,9.029509e+06,2.300720e+06,616198.277114,3.816651e+06,3.754515e+06,1.149754e+06,6.369422e+06,141156.384594,88981.403444,198717.888114
2,2025-01-03,2.374056e+06,748910.791885,4.023429e+06,1.218151e+06,4180.101787,2.335720e+06,8.928158e+05,-422558.369809,2.207848e+06,...,8.582433e+06,2.036586e+06,304436.686485,3.694728e+06,3.951604e+06,1.513097e+06,6.482908e+06,145447.565457,92267.027783,198626.036332
3,2025-01-04,2.303879e+06,712588.880896,4.040652e+06,1.236750e+06,113406.736142,2.360771e+06,8.075119e+05,-412573.786260,2.245207e+06,...,8.577951e+06,1.920855e+06,346486.960314,3.526409e+06,4.000211e+06,1.585604e+06,6.460590e+06,141490.585800,88125.000608,193306.262128
4,2025-01-05,2.226668e+06,612641.622197,3.944167e+06,9.993490e+05,-210116.067886,2.233631e+06,9.730704e+05,-337130.752564,2.250739e+06,...,8.417357e+06,1.966949e+06,356464.934201,3.623133e+06,3.654473e+06,1.048282e+06,6.407096e+06,145145.890306,94385.532989,197885.555892


In [21]:
import matplotlib.pyplot as plt
import os

script_dir = os.getcwd()
images_dir = os.path.join(os.path.dirname(script_dir), '../images')

if not os.path.exists(images_dir):
    os.makedirs(images_dir)

# Example for a single column
for column in columns_to_forecast:
    plt.plot(df_to_predict['Date_Of_Ownership'], df_to_predict[column], label='Actual')
    plt.plot(forecast_results[column]['ds'], forecast_results[column]['yhat'], label='Forecast')
    plt.fill_between(forecast_results[column]['ds'], forecast_results[column]['yhat_lower'], forecast_results[column]['yhat_upper'], alpha=0.2, label='Uncertainty Interval')
    plt.legend()
    plt.title(f"{column} prediction")
    file_path = os.path.join(images_dir, f"{column}.png")
    plt.savefig(file_path)

    plt.clf()
    
    # plt.show()

<Figure size 640x480 with 0 Axes>